In [57]:
import cv2
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [58]:
IMG_SIZE = 128
def get_cropped(img, final_size):
    '''
    Resize the image through its smallest dimension,
    then crop out the center and return as np array.
    final_size must be <= the min of image dimension
    '''
    size = img.size
    ratio = float(final_size) / min(size)
    new_image_size = tuple([int(x*ratio) for x in size])
    img = img.resize(new_image_size, Image.ANTIALIAS)
    width, height = img.size
    w_mid, h_mid = width // 2, height // 2
    img = np.array(img)
    img = img[(h_mid-final_size//2):(h_mid+final_size//2), (w_mid-final_size//2):(w_mid+final_size//2)]
    
    if img.shape == (final_size, final_size, 3):
        return img[:, :, 0]  # take only one channel of the image
    else:
        print('Error: Image size = {}'.format(size))

def load_images_from_dir(dirpath):
    images = []
    for filename in os.listdir(dirpath):
        try:
            img = Image.open(os.path.join(dirpath,filename))
            images.append(img)
        except:
            pass
    return images

def load_data(dirpath):
    '''Dont end the dirpath in a slash'''
    imgs = load_images_from_dir(dirpath)
    imgs = [get_cropped(img, IMG_SIZE) for img in imgs]
    imgs = [img for img in imgs if img is not None]
    labels = [int(os.path.basename(dirpath)) for _ in imgs]
    return imgs, labels

In [59]:
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, using LBP which is fast
    #there is also a more accurate but slow Haar classifier
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')

    #detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [60]:
subjects = ["", "One", "Two"]

# Dont end the dirpath in a slash
dirOne = 'gallaghar-cropped/8'
dirTwo = 'gallaghar-cropped/9'

faces1, labels1 = load_data(dirOne)  # images of subject One
faces2, labels2 = load_data(dirTwo)  # images of subject Two

faces = faces1 + faces2
labels = labels1 + labels2

Error: Image size = (412, 549)
Error: Image size = (499, 664)


In [61]:
# face_recognizer = cv2.face.LBPHFaceRecognizer_create()

face_recognizer = cv2.face.EigenFaceRecognizer_create()

# face_recognizer = cv2.face.FisherFaceRecognizer_create()

In [62]:
face_recognizer.train(faces, np.array(labels))

In [63]:
#function to draw rectangle on image 
#according to given (x, y) coordinates and 
#given width and heigh
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [64]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label= face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img

In [ ]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread('{}/794.jpg'.format(dirTwo))
test_img2 = cv2.imread('{}/678.jpg'.format(dirOne))

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
cv2.imshow(subjects[1], predicted_img1)
cv2.imshow(subjects[2], predicted_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()